## Reed-Muller符号

In [24]:
p = 2; m = 2
q = p**m
r = 1; m = 3
RM = codes.ReedMullerCode(GF(q), r, m)

生成行列

In [25]:
RM.generator_matrix()

4 x 64 dense matrix over Finite Field in z2 of size 2^2 (use the '.str()' method to see the entries)

パリティ検査行列

In [26]:
RM.parity_check_matrix()

60 x 64 dense matrix over Finite Field in z2 of size 2^2 (use the '.str()' method to see the entries)

符号長

In [27]:
RM.length()

64

符号の次元

In [28]:
RM.dimension()

4

最小距離

In [29]:
RM.minimum_distance()

48

符号語

In [30]:
list(RM)

[(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1),
 (z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2, z2),
 (z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 1, z2 + 

Rank metricへの拡張

In [31]:
codewords = list(RM)

In [ ]:
def vector_to_matrix(vec, rows, cols):
    return Matrix(GF(p), rows, cols, vec)
